In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

## 读入数据

In [4]:
data = pd.read_csv('fer2013.csv')

In [5]:
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


## 数据增强·

In [6]:
img_data = list(map(float,data.iloc[0,1].split()))
img_array = np.array(img_data).reshape(48,48)
img_array = img_array/255

In [9]:
M1 = cv2.getRotationMatrix2D((24,24),5,1.25)
M2 = cv2.getRotationMatrix2D((24,24),-5,1.25)
M3 = cv2.getRotationMatrix2D((24,24),5,0.85)
M4 = cv2.getRotationMatrix2D((24,24),-5,0.85)

In [10]:
img_width = 48
img_height = 48
img_channel = 1
num_process = 6
def process_data(data,label):
    num_data = len(data)
    img_process = np.zeros((num_data*num_process,img_height,img_width),dtype=np.float32)
    label_process = np.zeros((num_data*num_process),dtype=np.int32)
    for i in range(num_data):
        index = i*num_process
        img_data = list(map(float,data.iloc[i,1].split()))
        img_array = np.array(img_data).reshape(img_height,img_width)
        img_process[index] = img_array
        
        img_process[index+1] = cv2.flip(img_height,img_width)
        img_process[index] = img_array
        
        img_process[index+2] = cv2.warpAffine(img_array,M1,(48,48))
        img_process[index+3] = cv2.warpAffine(img_array,M2,(48,48))
        img_process[index+4] = cv2.warpAffine(img_array,M3,(48,48))
        img_process[index+5] = cv2.warpAffine(img_array,M4,(48,48))
        
        label_process[index:index+num_process] = label[i]
    
    return img_process,label_process

In [11]:
def translate_data(data_):
    result=[]
    for i in range(len(data_)):
        img_data = list(map(float,data_.iloc[i,1].split()))
        result.append(img_data)
    return np.array(result)

In [12]:
train_data = data[data['Usage']=="Training"]
val_data = data[data['Usage']=="PublicTest"]
test_data = data[data['Usage']=="PrivateTest"]

In [6]:
X_train = translate_data(train_data)
X_val = translate_data(val_data)
X_test = translate_data(test_data)

In [7]:
X_train = X_train/255.0
X_val = X_val/255.0
X_test = X_test/255.0

In [8]:
y_train = train_data['emotion'].values
y_val = val_data['emotion'].values
y_test = test_data['emotion'].values

In [9]:
y_train = np.eye(7,dtype=np.float32)[y_train]
y_val = np.eye(7,dtype=np.float32)[y_val]
y_test = np.eye(7,dtype=np.float32)[y_test]

In [10]:
y_test.shape

(3492, 7)

## 取mini-batch

In [11]:
index_batch = 0
m_train = X_train.shape[0]#28283
index_data = np.arange(m_train)

In [12]:
def next_batch(batch_size):
    global m_train,index_batch
    global X_train,y_train
    
    if index_batch == 0:
        np.random.shuffle(index_data)
        X_train = X_train[index_data]
        y_train = y_train[index_data]
    
    index_batch_end = index_batch+batch_size
    if index_batch_end > m_train:
        index_batch_end = m_train
    
    batch_x = X_train[index_batch:index_batch_end]
    batch_y = y_train[index_batch:index_batch_end]
    
    index_batch = index_batch_end%m_train
    
    return batch_x,batch_y

## 构建网络

In [13]:
#输入图像的长*宽*通道数
n_width = 48
n_height = 48
n_channel = 1
#输入图像数据大小
n_input = n_width*n_height*n_channel
#输出类别数
n_class = 7

In [14]:
#其他超参
num_epoch = 10
batch_size = 100
num_batches = int(train_data.shape[0]/batch_size)
learning_rate = 0.005

In [15]:
x = tf.placeholder(tf.float32,shape=(None,n_input),name='x')
x_ = tf.reshape(x,shape=[-1,n_height,n_width,n_channel])
y = tf.placeholder(tf.float32,shape=(None,n_class),name='y')

In [16]:
#Layer1
#input:batch*48*48*3
#filter:3*3*1*16
#conv_1: 48*48*16
#pool:24*24*16
w_1 = tf.Variable(tf.random_normal([3,3,n_channel,16]),dtype=tf.float32,name='w_1')
b_1 = tf.Variable(tf.random_normal([16]),dtype=tf.float32,name='b_1')
conv_1 = tf.nn.conv2d(x_,w_1,strides=[1,1,1,1],padding='SAME')+b_1
a_1 = tf.nn.relu(conv_1)

Instructions for updating:
Colocations handled automatically by placer.


In [17]:
pool_1 = tf.nn.max_pool(a_1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [18]:
#Layer2
#input:24*24*16
#filter:3*3*16*32
#conv_1: 24*24*32
#pool:12*12*32
w_2 = tf.Variable(tf.random_normal([3,3,16,32]),dtype=tf.float32,name='w_2')
b_2 = tf.Variable(tf.random_normal([32]),dtype=tf.float32,name='b_2')
conv_2 = tf.nn.conv2d(pool_1,w_2,strides=[1,1,1,1],padding='SAME')+b_2
a_2 = tf.nn.relu(conv_2)

In [19]:
pool_2 = tf.nn.max_pool(a_2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [20]:
#Layer3
#input:12*12*32
#filter:3*3*32*32
#conv_1: 12*12*32
w_3 = tf.Variable(tf.random_normal([3,3,32,32]),dtype=tf.float32,name='w_3')
b_3 = tf.Variable(tf.random_normal([32]),dtype=tf.float32,name='b_3')
conv_3 = tf.nn.conv2d(pool_2,w_3,strides=[1,1,1,1],padding='SAME')+b_3
a_3 = tf.nn.relu(conv_3)

In [21]:
#layer4
#input:7*7*32
#outpur:120
w_4 = tf.Variable(tf.random_normal([12*12*32,120]),dtype=tf.float32,name='w_4')
b_4 = tf.Variable(tf.random_normal([120]),dtype=tf.float32,name='b_4')
z_4 = tf.matmul(tf.reshape(a_3,[-1,12*12*32]),w_4)+b_4
a_4 = tf.nn.relu(z_4)

In [22]:
#layer5
#input:120
#outpur:84
w_5 = tf.Variable(tf.random_normal([120,84]),dtype=tf.float32,name='w_5')
b_5 = tf.Variable(tf.random_normal([84]),dtype=tf.float32,name='b_5')
z_5 = tf.matmul(a_4,w_5)+b_5
a_5 = tf.nn.relu(z_5)

In [23]:
#layer6
#input:84
#outpur:10
w_6 = tf.Variable(tf.random_normal([84,n_class]),dtype=tf.float32,name='w_6')
b_6 = tf.Variable(tf.random_normal([n_class]),dtype=tf.float32,name='b_6')
z_6 = tf.matmul(a_5,w_6)+b_6
y_hat = tf.nn.softmax(z_6)

## 代价函数

In [24]:
entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=z_6,labels=y)
cost = tf.reduce_mean(entropy)

In [25]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

## 准确率

In [26]:
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_hat,1))
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## 训练

In [27]:
cost_epoches = []
val_acc = []
with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    for epoch in range(num_epoch):
        for batch in range(num_batches):
            if batch%100==0:
                print(">>",end='')
            batch_x,batch_y = next_batch(batch_size)
            tfs.run(optimizer,feed_dict={x:batch_x,y:batch_y})
            cost_val = tfs.run(cost,feed_dict={x:batch_x,y:batch_y})
            cost_epoches.append(cost_val)   
        
        result = tfs.run(acc,feed_dict={x:X_val,y:y_val})
        val_acc.append(result)
        print("Epoch: {0:04d}, Val acc: {1:0.6f}".format(epoch,result))
    print("Model Trained.")
        
    test_acc = tfs.run(acc,feed_dict={x:X_test,y:y_test})
    print("Accuracy:",test_acc)

>>>>>>Epoch: 0000, Val acc: 0.207675
>>>>>>Epoch: 0001, Val acc: 0.225734
>>>>>>Epoch: 0002, Val acc: 0.216986
>>>>>>Epoch: 0003, Val acc: 0.187359
>>>>>>Epoch: 0004, Val acc: 0.176354
>>>>>>Epoch: 0005, Val acc: 0.156885
>>>>>>Epoch: 0006, Val acc: 0.213318
>>>>>>Epoch: 0007, Val acc: 0.177201
>>>>>>Epoch: 0008, Val acc: 0.211343
>>>>>>Epoch: 0009, Val acc: 0.187359
Model Trained.
Accuracy: 0.17583047


In [ ]:
plt.plot(cost_epoches)
plt.show()

In [ ]:
a = np.ones((3,3,3))

In [ ]:
a.shape

In [ ]:
b = np.array([10,20,30])

In [ ]:
b.shape

In [ ]:
c = a+b

In [ ]:
c.shape

In [ ]:
c[:,:,0]

In [ ]:
a = np.ones((1,25,25,1))

In [ ]:
b = tf.convert_to_tensor(a)

In [ ]:
c = tf.nn.max_pool(b,(1,2,2,1),(1,2,2,1),padding='SAME')

In [ ]:
with tf.Session() as tfs:
    d = tfs.run(c)

In [ ]:
d.shape